# Procesos iterativos

En este capítulo vamos a explorar algunos procesos iterativos que presentan un comportamiento interesante. Nos servirań como excusa para repasar los bucles y las operaciones con arrays.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Método de Newton

En la introducción del curso mostramos un método iterativo para calcular la raíz cuadrada. Se basaba en una idea intuitiva que funcionaba muy bien (hacer la media entre dos estimaciones). Pero cuando intentamos extenderla para calcular, por ejemplo, la raíz cúbica, el método converge mucho más lentamente.

In [ ]:
N = 27
x = 1

for _ in range(10):
    x = (x + N/x**2)/2
    print(x)

El [método de Newton](https://en.wikipedia.org/wiki/Newton%27s_method) nos dice cuál es la iteración óptima para resolver cualquier ecuación de la forma $f(x)=0$:

$$ x_{k+1} \leftarrow x_k - \frac{f(x)}{f'(x)}$$

La justificación es muy sencilla: en cada paso se encuentra la solución exacta a una aproximación lineal a $f(x)$. En la página de wikipedia hay una [animación](https://en.wikipedia.org/wiki/Newton%27s_method#Description) que lo explica muy bien.

Si la ecuación a resolver es $x^2 = N$, entonces $f(x) = x^2-N$ y $f'(x)=2x$, la expresión anterior es $x_{k+1} = (x_k + N/x_k)/2$. El método intuitivo era realmente el método de Newton.

Para la raíz cúbica tenemos $f(x) = x^3-N$ y $f'(x)=3x^2$, lo que da lugar a la regla $x_{k+1} \leftarrow (2 x_k + N/x_k^2 )/3$. Comprobamos que converge rápidamente:

In [ ]:
N = 27
x = 1

for _ in range(10):
    x = (2*x + N/x**2)/3
    print(x)

Veamos otro ejemplo: resolvamos la ecuación de Kepler 

$$x - b \sin(x)=a$$

para $b=0.3$ y $a=0.4$.

In [ ]:
a = 0.4
b = 0.3

def f(x):
    return x - b*np.sin(x) - a

def df(x):
    return 1 - b*np.cos(x)

x = 0.2

for _ in range(5):
    x = x - f(x)/df(x)
    print(x)

La convergencia es muy rápida. Si en vez de aplicar el método de Newton hacemos una iteración "ingenua" obtenida al despejar una $x$ de la ecuación también funciona pero mucho más lentamente.

In [ ]:
x = 0.2

for _ in range(10):
    x = a + b*np.sin(x)
    print(x)

Si la ecuación a resolver es complicada es conveniente usar funciones de derivación simbólica o numérica, para no tener que hacerlo a mano. Las herramientas de cálculo científico tienen funciones especializadas para esta tarea. En un capítulo posterior veremos las que proporciona Python.

## Mandelbrot

Una cuestión importante en los métodos interativos es determinar los valores iniciales que consiguen una secuencia convergente.

Consideremos una iteración muy simple como la siguiente:

$$z_{k+1} = z_k^2 + C$$

donde $z_0 =0$, para diferentes valores de C.

In [ ]:
C = 0.2

z = 0
for _ in range(10):
    z = z**2 + C
    print(z)

In [ ]:
C = 1.3

z = 0
for _ in range(10):
    z = z**2 + C
    print(z)

Con $C=0.2$ parece que la secuencia puede converger, o al menos no diverge, pero con $C=1.3$, claramente sí diverge.

No hay ningún problema en usar números complejos. De nuevo, dependiendo del valor de C la secuencia puede o no divergir.

In [ ]:
C = 0.2 + 0.3j

z = 0
for _ in range(10):
    z = z**2 + C
    print(z)

In [ ]:
C = 0.6 - 0.7j

z = 0
for _ in range(10):
    z = z**2 + C
    print(z)

El [conjunto de Mandelbrot](https://en.wikipedia.org/wiki/Mandelbrot_set) es el conjunto números complejos $C$ para los que la iteración $z\leftarrow z^2+C$ no diverge.

Podemos programarlo de la siguiente forma. Para un número razonable de iteraciones, si en algún momento $|z| > 2$ seguro que va a divergir, por lo que inmediatamente el resultado es falso y termina la función. Si se completa el bucle es probable que el valor de $C$ de partida esté dentro del conjunto.

In [ ]:
def mandel(c):
    z = 0
    for _ in range(30):
        z = z**2+c
        if abs(z) > 2:
            return False
    return True

In [ ]:
mandel(-0.5+0.2j)

In [ ]:
mandel(0.3+0.8j)

Para representarlo gráficamente evaluamos la función para todos los posibles valores de C en una región rectangular, con la resolución deseada. La función `imshow` permite representar matrices como imágenes, con diferentes "paletas" de color.

In [ ]:
plt.imshow( [[mandel(x + y*1j) for x in np.linspace(-1.5,0.5,100)] for y in np.linspace(-1,1,100)]);

Queda más bonito si la gráfica muestra con diferentes colores el número de pasos realizados. De esta forma se observa lo cerca que está cada punto al conjunto de Mandelbrot.

In [ ]:
def mandel(c):
    z = 0
    for k in range(30):
        z = z**2+c
        if abs(z) > 2:
            break
    return k

In [ ]:
mandel(0.5)

In [ ]:
mandel(-0.2+0.3j)

In [ ]:
plt.imshow( [[mandel(x + y*1j) for x in np.linspace(-1.5,0.5,100)] for y in np.linspace(-1,1,100)]);

Si queremos una imagen de mayor resolución el tiempo de cálculo empieza a ser apreciable.

Para que el cómputo sea más eficiente, en vez de usar las operaciones normales de Python que operan sobre números individuales es preferible usar operaciones aritméticas sobre arrays de `numpy`. Realizaremos cada paso de iteración para todos los C "a la vez".

Para crear una matriz con todos los valores de C podemos hacer lo siguiente. Definimos el rango de valores que queremos visualizar en cada eje (centrado en la posición deseada y con un cierto ancho D) y luego formamos un array a partir de una lista de listas.

In [ ]:
n = 1000

centro = -0.5; D=1
#c = -0.5+0.6j; D = 0.2

rx = np.linspace(centro.real-D,centro.real+D,n)
ry = np.linspace(centro.imag-D,centro.imag+D,n)[::-1]

In [ ]:
%%time

C = np.array([[x+y*1j for x in rx] for y in ry])

La operación es muy lenta. La lista de listas contiene un millón del elementos que hay que crear uno por uno. Es mucho más rápido fabricar la matriz C de forma directa usando la función `meshgrid`, que explicamos más adelante (o combinando una matriz fila y una matriz columna, aprovechando el *automatic broadcasting*).

In [ ]:
n = 1000

centro = -0.5; D=1
#c = -0.5+0.6j; D = 0.2

rx = np.linspace(centro.real-D,centro.real+D,n)
ry = np.linspace(centro.imag-D,centro.imag+D,n)[::-1]

In [ ]:
x,y = np.meshgrid(rx,ry)

In [ ]:
C = x + y*1j

Ahora podemos aplicar la iteración a todos los puntos de partida y mostrar los que no divergen.

In [ ]:
Z = np.zeros([n,n])

for k in range(30):
    Z = Z**2 + C

plt.figure(figsize=(10,10))
plt.imshow( abs(Z)<2, 'gray'); plt.axis('off');

Algunas versiones de numpy producen avisos ("warnings") indicando que se ha producido un "overflow" en algunos elementos. Son magnitudes mayores que la capacidad del tipo `float`. Para evitarlo usaremos una "máscara" S (una matriz de valores lógicos), que indicará los elementos que se mantienen acotados. Esto nos permitirá también fabricar fácilmente el gráfico con colores según el número de iteraciones. Para ello usaremos una matriz N que irá contando en cada posición los pasos realizados. Observa que las operaciones de iteración en Z y de incremento de N se efectúan solo en las posiciones donde la máscara S es verdadera.

In [ ]:
Z = np.zeros([n,n],complex)
N = np.zeros([n,n])

for k in range(30):
    S = abs(Z) < 2
    Z[S] = (Z**2 + C)[S]
    N[S] += 1

plt.figure(figsize=(10,10))
plt.imshow(N, 'coolwarm', extent=[rx[0],rx[-1],ry[-1],ry[0]]);
plt.title('conjunto de Mandelbrot'); plt.xlabel('real'); plt.ylabel('imag');

Un ejercicio interesante es definir una función que crea la imagen del conjunto de mandelbrot en una posición y zoom deseados y usarla para hacer una animación.

## Cuencas de atracción

Más arriba hemos resuelto $x^3=N$ en el dominio real y hemos visto que el método de Newton converge rápidamente a la solución. Resulta interesante investigar los puntos de partida que convergen a cada una de las soluciones de la sencilla ecuación $z^3=1$ en el dominio complejo.

Las soluciones se reparten en el círculo unidad:

In [ ]:
z1 = np.exp(1j * 0*2*np.pi/3)
z2 = np.exp(1j * 1*2*np.pi/3)
z3 = np.exp(1j * 2*2*np.pi/3)
print(z1,z2,z3)

Podría pensarse que el metodo de Newton converge a la solución más próxima al punto de partida. Comprobémoslo. Generamos una matriz con una malla de puntos de partida para aplicar el método con todos los elementos a la vez usando operaciones sobre arrays.

In [ ]:
n = 1000
r = np.linspace(-1,1,n)
x0,y0 = np.meshgrid(r,r)
z0 = x0+y0*1j

In [ ]:
def cube(z):
    return z**3-1

def dcube(z):
    return 3*z**2

In [ ]:
#%%time

Z = z0

for k in range(10):
    Z = Z - cube(Z)/dcube(Z)

Para dibujar la cuenca fabricamos máscaras que indican los elementos que al final de la iteración están más cerca de cada raíz. Estas máscaras nos permiten asignar colores (con la codificación RGB, Red Green Blue) a la matriz mapa, para mostrar las diferentes regiones con `imshow`.

In [ ]:
eps = 1e-3

m1 = abs(Z-z1) < eps
m2 = abs(Z-z2) < eps
m3 = abs(Z-z3) < eps

mapa = np.zeros([n,n,3])

mapa[m1] = (1,0,0)
mapa[m2] = (0,1,0)
mapa[m3] = (0,0,1)

plt.figure(figsize=(10,10))
plt.imshow(mapa);

Las cuencas de convergencia a cada solución tienen una frontera muy compleja ([Newton fractal](https://en.wikipedia.org/wiki/Newton_fractal)).

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(~(m1 | m2 | m3), 'gray');

Aquí hay un interesante [blog post](https://web.archive.org/web/20230201164602/http://www.investigacionyciencia.es/blogs/matematicas/33/posts/derivadas-y-fractales-de-newton-11288) sobre este tema.

## meshgrid *

Considera el producto cartesiano de dos contenedores:

In [ ]:
[(x,y) for x in [1,2,3,4] for y in [10,20,30] ]

Podemos expresarlo como una lista de listas, similar a una matriz.

In [ ]:
[[(x,y) for x in [1,2,3,4]] for y in [10,20,30] ]

La función `meshgrid` de `numpy` fabrica un producto cartesiano de forma muy eficiente, pero nos da los elementos por separado:

In [ ]:
x,y = np.meshgrid([1,2,3,4],[10,20,30])

In [ ]:
x

In [ ]:
y

Esto tiene múltiples aplicaciones. Permite dibujar superficies 3D, o, como hemos visto más arriba, preparar un conjunto de valores C para el conjunto de Mandelbrot o de valores iniciales para el método de Newton.

In [ ]:
x+y*1j

Otra forma de conseguir el mismo resultado es combinar una matriz fila y una matriz columna, aprovechando el *automatic broadcasting*

In [ ]:
x = np.array([[1,2,3,4]])
y = np.array([[10,20,30]]).T

x + y*1j

## Autómata celular

Un proceso iterativo que se hizo muy famoso es el [Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life).
Se trata de un conjunto de células dispuestas en una parrilla bidimensional, cada una de las cuales se encuentra en cada momento en un estado "vivo" o "muerto". El tiempo avanza en pasos discretos y en cada instante una célula puede nacer, continuar viviendo, o morir, dependiendo del número de células vivas que hay en su entorno. Los parámetros típicos que producen un comportamiento interesante son los siguientes: una célula nace en un sitio si en su entorno hay 3 vecinos; una célula muere si en su entorno hay menos de 2 (está muy sola) o más de 3 vecinos (no hay alimento para todas). 

La implementación tradicional es la siguiente, basada en 4 bucles anidados. El estado siguiente se irá guardando en un array auxiliar `r`. Para evitar que los bordes del mundo sean posiciones especiales, vamos a trabajar con un mundo ilimitado "toroidal" o cíclico que se cierra sobre sí mismo. Esto se programa muy fácilmente tomando el resto de la división de las coordenadas entre el número de celdas en cada dimensión.

In [ ]:
import numpy as np

s1 = 2; s2 = 3
n1 = 3; n2 = 3

def life(w):
    n,m = w.shape
    r = np.empty_like(w)
    for i in range(n):
        for j in range(m):
            v = 0
            for k in [-1,0,1]:
                for l in [-1,0,1]:
                    if w[(i+k)%n, (j+l)%n]:
                        v += 1
            if w[i,j]:
                v -= 1
                r[i,j] = s1 <= v <= s2
            else:
                r[i,j] = n1 <= v <= n2
    return r

In [ ]:
n = 10
x = 1*(np.random.rand(n,n) > 0.6);

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(x, 'gray'); plt.title('instante actual')
plt.subplot(1,2,2)
plt.imshow(life(x),'gray'); plt.title('instante siguiente');

Creamos una animación partiendo de una configuración aleatoria, pero borrando la zona superior izquierda para meter un "glider".

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='html5')
from IPython.display import HTML

In [ ]:
n = 50
x = 1*(np.random.rand(n,n) > 0.6);

x[:25,:25] = 0
x[5:8,5:8] = [[0,1,0],
              [0,0,1],
              [1,1,1]]


fig, ax = plt.subplots(figsize=(5,5))
plt.close();

img = ax.imshow(1-x,'gray')

def animate(k):
    global x
    img.set_data(1-x)
    x = life(x)
    return [img]

fps=10
T = 5
ani = animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=T*fps, interval=1000/fps, blit=True, repeat=False)
HTML(ani.to_jshtml())

Es curioso que una regla tan sencilla dé lugar a configuraciones de células con comportamientos no triviales (osciladores, estructuras caminantes, cañones que disparan proyectiles, etc.). De hecho, es posible simular cualquier computación con un autómata celular.

En lenguajes "compilados" como Fortran o C una definición como la anterior sería muy eficiente. Pero en lenguajes "interpretados" como Python o R es mucho mejor evitar los bucles explícitos y utilizar operaciones sobre arrays en bloque que están optimizadas.

En un "mundo" más grande de $400^2$ celdas la definición anterior necesita aproximadamente 1s para calcular un paso de evolución (depende del ordenador).

In [ ]:
n = 400
x = 1*(np.random.rand(n,n) > 0.6);

In [ ]:
%%time

for _ in range(10):
    x = life(x)

Una implementación alternativa con operaciones de `numpy` es la siguiente. La idea es calcular los vecinos creando un array 3D con los 8 desplazamientos en las dos direcciones de modo que sumando los elementos a lo largo del eje adecuado conseguimos el número de vecinos de cada celda. Finalmente creamos el resultado con operaciones lógicas elemento a elemento.

In [ ]:
def vecinos(w):
    return np.array([np.roll(np.roll(w,k,axis=1),l,axis=0)
                     for k in [-1,0,1] 
                     for l in [-1,0,1] if (k,l) != (0,0) ]
                   ).sum(axis=0)

def life_a(w):
    v = vecinos(w)
    return np.where(w, (s1 <=v)&(v<=s2) , (n1<=v)&(v<=n2) )

Comprobamos que esta definición es equivalente a la anterior:

In [ ]:
np.all(life(x) == life_a(x))

Pero es más de 200 veces más rápida.

In [ ]:
%%time

for _ in range(10*200):
    x = life_a(x)